# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [ ]:
import os  #serve para manipular caminhos de diretorios e arquivos
import io  #lida com fluxos de arquivos
import numpy 
import pandas as pd  
from pandas import DataFrame  
from sklearn.feature_extraction.text import CountVectorizer  #transformar mensagens em valores numéricos
from sklearn.naive_bayes import MultinomialNB  #algoritimo responsavel pelo aprendizado de maquina para classificacao


def readFiles(path):
    for root, dirnames, filenames in os.walk(path):  #caminha pelos diretorios especificados
        for filename in filenames:  #itera sobre cada arquivo
            path = os.path.join(root, filename)  #pega o caminho completo do arquivo

            inBody = False  #identificar se estamos na parte do corpo do e-mail
            lines = []  #lista para armazenar as linhas do corpo do e-mail
            f = io.open(path, 'r', encoding='latin1')  # Abre o arquivo 
            for line in f:
                if inBody:  # caso ja esteja na parte do corpo, adicionamos as linhas
                    lines.append(line)
                elif line == '\n':  # Uma linha em branco separa o cabecalho do corpo
                    inBody = True
            f.close()  # Fecha o arquivo
            message = '\n'.join(lines)  # Junta as linhas do corpo em uma unica string
            yield path, message  #retorna o caminho do arquivo e o conteudo do e-mail



def dataFrameFromDirectory(path, classification):
    rows = []  #lista para armazenar os dados de cada arquivo
    index = []  #lista para armazenar os nomes dos arquivos como indice
    for filename, message in readFiles(path):  # Usa a função anterior para ler arquivos
        rows.append({'message': message, 'class': classification})  # Adiciona uma linha com a mensagem e a classe ('spam' ou 'ham')
        index.append(filename)  # Adiciona o nome do arquivo ao indice anteriormente citado
        

    return DataFrame(rows, index=index)  # Retorna um DataFrame com os dados


data = DataFrame({'message': [], 'class': []})  # Cria um DataFrame vazio com colunas 'message' e 'class'

#For Pandas 1.3:
data = pd.append(data, dataFrameFromDirectory('/Users/heloamillergrassi/Downloads/MLCourse/emails/spam','spam'))
data = pd.append(data, dataFrameFromDirectory('/Users/heloamillergrassi/Downloads/MLCourse/emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [ ]:
data.head()

Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [ ]:
#basicamnete conta quantas vezes certa palavra apareceu nos emails

#transformar as mensagens em uma matriz numérica
vectorizer = CountVectorizer()

# 'fit_transform' aprende o vocabulário das mensagens e transforma as mensagens em uma matriz de contagem de palavras
counts = vectorizer.fit_transform(data['message'].values)

# Criando o classificador MultinomialNB (Naive Bayes para dados de contagem)
classifier = MultinomialNB()

# Definindo as classes (spam ou ham) como o alvo
targets = data['class'].values

# Treinando o classificador com as contagens de palavras e os alvos
classifier.fit(counts, targets)


Let's try it out:

In [ ]:
#conta as palavras do email posterior, para conseguirmos definir se e spam ou nao

# Novas mensagens de exemplo
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]

# Transformando as novas mensagens de email em uma representação numeica
example_counts = vectorizer.transform(examples)

# Fazendo a previsao das classes (spam ou ham) para as novas mensagens
predictions = classifier.predict(example_counts)

# Exibindo as previsoes
print(predictions)


## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [ ]:
import os  #serve para manipular caminhos de diretorios e arquivos
import io  #lida com fluxos de arquivos
import numpy 
import pandas as pd  
from pandas import DataFrame  
from sklearn.feature_extraction.text import CountVectorizer  #transformar mensagens em valores numéricos
from sklearn.naive_bayes import MultinomialNB  #algoritimo responsavel pelo aprendizado de maquina para classificacao


def readFiles(path):
    for root, dirnames, filenames in os.walk(path):  #caminha pelos diretorios especificados
        for filename in filenames:  #itera sobre cada arquivo
            path = os.path.join(root, filename)  #pega o caminho completo do arquivo

            inBody = False  #identificar se estamos na parte do corpo do e-mail
            lines = []  #lista para armazenar as linhas do corpo do e-mail
            f = io.open(path, 'r', encoding='latin1')  # Abre o arquivo 
            for line in f:
                if inBody:  # caso ja esteja na parte do corpo, adicionamos as linhas
                    lines.append(line)
                elif line == '\n':  # Uma linha em branco separa o cabecalho do corpo
                    inBody = True
            f.close()  # Fecha o arquivo
            message = '\n'.join(lines)  # Junta as linhas do corpo em uma unica string
            yield path, message  #retorna o caminho do arquivo e o conteudo do e-mail



def dataFrameFromDirectory(path, classification):
    rows = []  #lista para armazenar os dados de cada arquivo
    index = []  #lista para armazenar os nomes dos arquivos como indice
    for filename, message in readFiles(path):  # Usa a função anterior para ler arquivos
        rows.append({'message': message, 'class': classification})  # Adiciona uma linha com a mensagem e a classe ('spam' ou 'ham')
        index.append(filename)  # Adiciona o nome do arquivo ao indice anteriormente citado
        

    return DataFrame(rows, index=index)  # Retorna um DataFrame com os dados


data = DataFrame({'message': [], 'class': []})  # Cria um DataFrame vazio com colunas 'message' e 'class'
data = pd.append(data, dataFrameFromDirectory('/Users/heloamillergrassi/Downloads/MLCourse/emails/spam','spam'))
data = pd.append(data, dataFrameFromDirectory('/Users/heloamillergrassi/Downloads/MLCourse/emails/ham', 'ham'))

# Criando o vetor de características a partir das mensagens
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['message'].values)  # Transforma as mensagens em uma matriz de contagem de palavras

# Definindo as classes (spam ou ham) como alvo
y = data['class'].values

# Dividindo os dados em conjunto de treino e conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criando o classificador Multinomial Naive Bayes
classifier = MultinomialNB()

# Treinando o classificador com os dados de treino
classifier.fit(X_train, y_train)

# Fazendo previsões com os dados de teste
y_pred = classifier.predict(X_test)

# Avaliando a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

print(f'\accuracy do modelo: {accuracy:.4f}')
